In [1]:
#Import Matplotlib Libraries
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
#Import Numpy, Pandas, and Datetime Libraries
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [4]:
# create path to hawaii sqlite database
hawaii_path = "Resources/hawaii.sqlite"

In [5]:
# create engine and connection to hawaii.sqlite
engine = create_engine(f"sqlite:///{hawaii_path}")
conn = engine.connect()

In [6]:
#Declare a base using autobase_map
Base = automap_base()

In [7]:
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

In [8]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [9]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [10]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [ ]:
#Preview Measurement Database in a Pandas Dataframe
measurement_df = pd.read_sql("SELECT * FROM Measurement", conn)
measurement_df.head()

In [ ]:
#Preview Station Database in a Pandas Dataframe
station_df = pd.read_sql("SELECT * FROM Station", conn)
station_df.head()

In [ ]:
#Create inspector and connect it to the engine
inspector = inspect(engine)

In [ ]:
#Use inspector to print the column names in the measurement table
columns = inspector.get_columns('measurement')
for column in columns:
    print(column["name"], column["type"])

In [ ]:
#Use inspector to print the column names in the station table
columns = inspector.get_columns('station')
for column in columns:
    print(column["name"], column["type"])

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
recent_date

In [ ]:
#Find date 1 year prior to most recent date
year_ago_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
year_ago_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results.

# Perform a query to retrieve the data and precipitation scores and save query as a Pandas DF
data_df = pd.read_sql("SELECT date, prcp FROM Measurement WHERE date >= '2016-08-23'", conn)
data_df = data_df.set_index('date')
data_df.head()

In [ ]:
#Plot the data
data_df.plot(color="blue", figsize=(15,10))
plt.xlabel('Date')
plt.ylabel('Inches of Rain')
plt.title('Precipitation Data')

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
data_df['prcp'].describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
number_of_stations = session.query(Station).count()
print(f"There are {number_of_stations} stations in the dataset")

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
list_of_stations = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
list_of_stations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
active_station_info = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').all()
active_station_info

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
temp_data_12mos = session.query(Measurement.tobs).\
    filter(Measurement.station == 'USC00519281').\
    filter(Measurement.date >= year_ago_date).all()
df = pd.DataFrame(temp_data_12mos, columns=['tobs'])
df.plot.hist(bins=12)
plt.xlabel("Temperature")
plt.title('Last 12 Months Temperature Data')


# Close session

In [ ]:
# Close Session
session.close()